### Script to generate FATES parameters using LHS

In [1]:
from scipy.stats import qmc
import numpy as np

import csv
import pandas as pd
import os
import netCDF4 as nc4
import sys
import shutil
from tempfile import TemporaryFile                                                                                                                                 
import argparse                                                                                                                                                                                                                                                                                                       
import tempfile 
import random
import re

import functions_py.modp as mp


ImportError: cannot import name qmc

In [ ]:
random.seed(32)

#### Read in min and max values for each parameter and pft 

In [ ]:
param_ranges_full = pd.read_csv('/global/homes/j/jneedham/DBEN/Parameter_files/BCI_ensemble_params_trimmed.csv')
param_ranges = param_ranges_full[['param', 'value_min', 'value_max', 'pft', 'organ']]

# number of parameters
n_params = len(param_ranges)

# number of PFTs - some are global so subtract one
n_pfts = len(pd.unique(param_ranges['pft'])) - 1

param_names = list(param_ranges.param)
pfts = list(param_ranges.pft)
organs = list(param_ranges.organ)

print(param_ranges)

In [ ]:
n_inst = 120

sampler = qmc.LatinHypercube(d=n_params)
sample = sampler.random(n=n_inst)

# scale to parameter ranges
l_bounds = param_ranges['value_min']
u_bounds = param_ranges['value_max']

scaled_sample = qmc.scale(sample, l_bounds, u_bounds)

print(scaled_sample[0,:])

In [ ]:
## Read in defaut FATES file - note that this is the default for FATES but with:
# - updated allometries for tropical PFTs
# - size bins that are consistent with the DBEN protocol. 
# - supplemental seed rain
# - updated vai bins
# - atkin respiration 
# - size dependent mortality 

input_fname = '/global/homes/j/jneedham/DBEN/Parameter_files/fates_params_bci_base.nc'

# for each sample 
for i in range(0,n_inst) :
    
    # final parameter file name
    fout = '/global/homes/j/jneedham/DBEN/Parameter_files/Ensemble_params/BCI/fates_params_bci_ens_{0}.nc'.format(i)
    
    shutil.copyfile(input_fname, fout)                                                                                                                             
   
    # loop through each parameter and apply either to the correct pft or globally
    for j in range(0, n_params) : 
        
        var = param_names[j]
        pft = pfts[j]
        organ = organs[j]
        
        val = scaled_sample[i, j]
        
        mp.main(var = var, pft = pft, fin = fout, val = val, 
                    fout = fout, O = 1, organ = organ)
        
        if var == 'fates_mort_bmort' and pft == 1 : 
            pft = pft + 1
            val = val * 0.75
            mp.main(var = var, pft = pft, fin = fout, val = val, 
                    fout = fout, O = 1, organ = organ)
            
        if var == 'fates_wood_density' and pft == 1 : 
            pft = pft + 1
            val = val * 1.5
            mp.main(var = var, pft = pft, fin = fout, val = val, 
                    fout = fout, O = 1, organ = organ)
            
        if var == 'fates_leaf_vcmax25top' and pft == 1 : 
            pft = pft + 1
            val = val * 0.9
            mp.main(var = var, pft = pft, fin = fout, val = val, 
                    fout = fout, O = 1, organ = organ)

            
    

In [ ]:
print(scaled_sample.shape)

In [ ]:
tmp = scaled_sample[0:50,:]

df  = pd.DataFrame(data=tmp, columns = param_names)

